In [ ]:
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.models import Model, load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir = "/content/drive/MyDrive/attractiveness/Testing/train"
test_dir = "/content/drive/MyDrive/attractiveness/Testing/test"
val_dir = "/content/drive/MyDrive/attractiveness/Testing/valid"

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,                                                                                                                   
                                                           rotation_range=0.2,
                                                           width_shift_range=0.2,
                                                           height_shift_range=0.2,
                                                           zoom_range = 0.2, 
                                                           horizontal_flip=True)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)

val_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)

In [ ]:
#batch_size = 20

train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           #batch_size=batch_size,
                                           class_mode = "binary"
                                           )
test_data = test_gen.flow_from_directory(test_dir,
                                         target_size = (224,224),
                                         #batch_size=batch_size,
                                         class_mode = "binary"
                                         )
val_data = val_gen.flow_from_directory(val_dir,
                                       target_size = (224,224),
                                       #batch_size=batch_size,
                                       class_mode = "binary"
                                       )

Found 3717 images belonging to 2 classes.
Found 250 images belonging to 2 classes.
Found 298 images belonging to 2 classes.


In [ ]:
labels = list(train_data.class_indices.keys())

labels

['average', 'beautiful']

In [ ]:
# visualizing the train data

import matplotlib.pyplot as plt

plt.figure(figsize=(16,16))

for i in range(18):
  image,label = train_data.next()

  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(labels[int(label[i])])
  plt.axis("off")

In [ ]:
from tensorflow.keras import layers

model = tf.keras.Sequential([
                             layers.Conv2D(filters= 62, kernel_size= 2, activation="relu", input_shape=(224,224,3)),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 62, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 62, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Flatten(),

                             layers.Dense(128, activation= "relu"),
                             layers.Dropout(0.5),

                             layers.Dense(128, activation= "relu"),
                             layers.Dropout(0.5),

                             layers.Dense(1, activation= "sigmoid")
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 223, 223, 62)      806       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 62)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 110, 110, 62)      15438     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 55, 55, 62)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 54, 54, 62)        15438     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 27, 27, 62)       0

In [ ]:
# compiling the model

model.compile(
    loss = tf.keras.losses.binary_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"],
    run_eagerly=True
)

In [ ]:
model.fit(train_data,
          epochs = 15,
          steps_per_epoch = len(train_data),
          validation_data = val_data,
          validation_steps = len(val_data)
          )

Epoch 1/15
117/117 [==============================] - 580s 5s/step - loss: 0.5508 - accuracy: 0.7471 - val_loss: 0.5613 - val_accuracy: 0.7550
Epoch 2/15
117/117 [==============================] - 54s 463ms/step - loss: 0.4982 - accuracy: 0.7778 - val_loss: 0.6290 - val_accuracy: 0.7047
Epoch 3/15
117/117 [==============================] - 55s 471ms/step - loss: 0.4740 - accuracy: 0.7953 - val_loss: 0.4761 - val_accuracy: 0.8054
Epoch 4/15
117/117 [==============================] - 55s 468ms/step - loss: 0.4416 - accuracy: 0.8076 - val_loss: 0.4492 - val_accuracy: 0.8087
Epoch 5/15
117/117 [==============================] - 55s 467ms/step - loss: 0.4167 - accuracy: 0.8251 - val_loss: 0.3696 - val_accuracy: 0.8490
Epoch 6/15
117/117 [==============================] - 55s 468ms/step - loss: 0.3439 - accuracy: 0.8577 - val_loss: 0.2997 - val_accuracy: 0.8859
Epoch 7/15
117/117 [==============================] - 55s 473ms/step - loss: 0.2906 - accuracy: 0.8849 - val_loss: 0.1532 - val_accu

In [ ]:
model_evaluation = model.evaluate(test_data)

8/8 [==============================] - 137s 20s/step - loss: 0.0507 - accuracy: 0.9880


In [ ]:
print(f"Model Accuracy: {model_evaluation[1] * 100 : 0.2f} %")

Model Accuracy:  98.80 %


In [ ]:
model.save('/Train_Test_Clean.h5')

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
plt.figure(figsize=(16,16))

#for i in range(18):

  #image, label = test_data.next()
model_pred = model.predict(test_data) 


  #plt.subplot(6,3,i+1)
  #plt.imshow(image[i])
  #plt.title(f"Prediction : {labels[int(tf.round(model_pred[i]))]}\n Original: {labels[int(label[i])]}")
  #plt.subplots_adjust(top= 1.25)
  #plt.axis("off")

<Figure size 1152x1152 with 0 Axes>

In [ ]:
model_pred[1][0]

0.04186619

In [ ]:
ans = []
for i in model_pred:
  print (i[0])
  if i[0] < 0.5:
    ans.append(0)
  else:
    ans.append(1)

In [ ]:
ans

In [ ]:
from sklearn.metrics import confusion_matrix
print('Confusion Matrix')
cm = confusion_matrix(test_data.classes, ans)
cm

Confusion Matrix


array([[42, 58],
       [57, 93]])

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

print('Accuracy Score',accuracy_score(test_data.classes, ans)*100,'%')
print('Precision Macro Score',precision_score(test_data.classes, ans,average = 'macro')*100,'%')
print('Recall_Score',recall_score(test_data.classes, ans, average = 'macro')*100,'%')
print('F_Score',f1_score(test_data.classes, ans, average = 'macro')*100,'%')

Accuracy Score 54.0 %
Precision Macro Score 52.006823198876184 %
Recall_Score 52.0 %
F_Score 52.00253760496837 %
